# Main notebook - experimenting with guesses

SPDX-License-Identifier: 0BSD

This does semantic search based solely on repository name and user input.

Compare to
[`findrepo`](https://github.com/EliahKagan/newrepo-findrepo#using-findrepo).

In [1]:
import logging

import numpy as np
from tabulate import tabulate

import fr2ex

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
names = fr2ex.remote.fetch_repo_names()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [4]:
len(names)

721

In [5]:
moderation = fr2ex.moderation.get_moderation(names)

INFO:root:Querying OpenAI moderation endpoint.


In [6]:
len(moderation)

721

In [7]:
any(fr2ex.moderation.any_flagged(result) for result in moderation)

False

In [8]:
fr2ex.tokens.report_cost(names)

It looks like the rate is $0.0001 per 1000 tokens. If so, the cost to
process 2053 tokens is about $0.0002053 (that is, 0.0205300 cents).


In [9]:
embeddings = fr2ex.embedding.embed_many(names)

INFO:root:Querying OpenAI embeddings endpoint.


In [10]:
def guess(name: str, count: int = 5) -> None:
    """Show top guesses for similarity of name to already embedded names."""
    scores = embeddings @ fr2ex.embedding.embed(name)
    ordering = sorted(zip(scores, names), reverse=True)
    table = [(name, score) for score, name in ordering]
    return tabulate(table[:count], tablefmt='html', floatfmt='.6f')

In [11]:
guess('algorithm')

Calculator,0.868892
Geometry,0.865471
algorithms-suggestions,0.860213
Alias,0.851718
Benchmark,0.849351


In [12]:
guess('sorting')

SortingRanges,0.932455
Sorts,0.909035
sortkey,0.859505
PartialSort,0.854997
tsort,0.840392


In [13]:
guess('algorithm-visualization')

algorithms-suggestions,0.889127
algorithms-python,0.859428
crystal-algo,0.855611
Graph-gist,0.842344
codegraph,0.832857


In [14]:
guess('maven')

try-maven,0.916256
mavener,0.880057
retry-maven,0.868163
pimpl,0.836303
try-gradle,0.833540


In [15]:
guess('find-repo')

newrepo-findrepo,0.916323
findrepo2-experiment,0.867370
backup-repos,0.865011
find-articles,0.856997
TestRepo,0.842254


In [16]:
guess('graph theory', count=10)

codegraph,0.881024
Graph,0.879386
Graph-gist,0.864565
Geometry,0.839661
wpf-graph,0.829286
graphyaml,0.820479
gh-profile,0.814717
c-sketches,0.813789
trig,0.809130
Draw,0.808729


In [17]:
guess('shortest paths', count=10)

Dijkstra,0.891185
Paths,0.838848
TreeTraversal,0.830707
Kruskal,0.816533
algorithms-suggestions,0.816335
Traverse,0.813205
Permutations,0.806636
treejs,0.801389
DFS,0.800918
TreeTraversalAnimations,0.800318


In [18]:
guess('shortest-paths', count=10)

Dijkstra,0.891988
Paths,0.836293
TreeTraversal,0.836137
Traverse,0.819954
algorithms-suggestions,0.819141
Kruskal,0.814023
type-arrows,0.812781
kt-short,0.811081
sequence-dag,0.809232
dot-and-cross,0.806380


In [19]:
guess('graph traversal', count=10)

TreeTraversal,0.897469
Traverse,0.870004
Graph,0.868454
codegraph,0.856849
Graph-gist,0.850980
TraverseTree,0.850101
TreeTraversalAnimations,0.845119
Dijkstra,0.824611
DFS,0.823564
sequence-dag,0.819539


In [20]:
guess('graph-traversal', count=10)

TreeTraversal,0.904177
Traverse,0.879879
Graph,0.878466
Graph-gist,0.875128
codegraph,0.872133
TraverseTree,0.865409
TreeTraversalAnimations,0.851328
Dijkstra,0.842215
wpf-graph,0.841237
sequence-dag,0.837101


In [21]:
guess('graphics', count=10)

Graph,0.889388
printing,0.883153
Geometry,0.881506
Draw,0.875333
fps,0.859527
works,0.848995
coordinates,0.845295
typevars,0.843457
Components,0.842692
interop,0.841446


In [22]:
guess('deluge')

Flood,0.865039
foobar,0.826051
lambda-demo,0.819182
Dynamo,0.816846
yule,0.814101


In [23]:
guess('source control')

git,0.822911
Linkage,0.807101
gitscripts,0.806857
capture.net,0.803123
test-site,0.801839


In [24]:
guess('source-control')

git,0.831433
git-notes,0.830786
example-submodule,0.818571
dot-imports,0.815824
gitscripts,0.813118
